# Gitlab 工具包

`Gitlab` 工具包包含允许 LLM 代理与 gitlab 存储库进行交互的工具。
该工具是 [python-gitlab](https://github.com/python-gitlab/python-gitlab) 库的封装。

## 快速入门
1. 安装 python-gitlab 库
2. 创建一个 Gitlab 个人访问令牌
3. 设置你的环境变量
4. 使用 `toolkit.get_tools()` 将工具传递给你的代理

下面将详细解释每一步。

1. **获取 Issue** - 从存储库获取 issue。

2. **获取 Issue 详情** - 获取关于特定 issue 的详细信息。

3. **评论 Issue** - 在特定 issue 上发布评论。

4. **创建合并请求** - 从机器人的工作分支创建合并请求到基础分支。

5. **创建文件** - 在存储库中创建新文件。

6. **读取文件** - 从存储库读取文件。

7. **更新文件** - 更新存储库中的文件。

8. **删除文件** - 从存储库中删除文件。

## 设置

### 1. 安装 `python-gitlab` 库

In [ ]:
%pip install --upgrade --quiet  python-gitlab langchain-community

### 2. 创建一个 Gitlab 个人访问令牌

请[在此处遵循说明](https://docs.gitlab.com/ee/user/profile/personal_access_tokens.html)创建 Gitlab 个人访问令牌。确保您的应用程序具有以下仓库权限：

* read_api
* read_repository
* write_repository

### 3. 设置环境变量

在初始化您的代理之前，需要设置以下环境变量：

* **GITLAB_URL** - 托管 Gitlab 的 URL。默认为 "https://gitlab.com"。
* **GITLAB_PERSONAL_ACCESS_TOKEN**- 您在上一步中创建的个人访问令牌
* **GITLAB_REPOSITORY**- 您希望您的机器人操作的 Gitlab 仓库名称。必须遵循 \{username\}/\{repo-name\} 的格式。
* **GITLAB_BRANCH**- 机器人将进行提交的分支。默认为 'main'。
* **GITLAB_BASE_BRANCH**- 您仓库的基础分支，通常是 'main' 或 'master'。合并请求将从此分支创建。默认为 'main'。

## 示例：简单代理

In [3]:
import os

from langchain.agents import AgentType, initialize_agent
from langchain_community.agent_toolkits.gitlab.toolkit import GitLabToolkit
from langchain_community.utilities.gitlab import GitLabAPIWrapper
from langchain_openai import OpenAI

In [53]:
# Set your environment variables using os.environ
os.environ["GITLAB_URL"] = "https://gitlab.example.org"
os.environ["GITLAB_PERSONAL_ACCESS_TOKEN"] = ""
os.environ["GITLAB_REPOSITORY"] = "username/repo-name"
os.environ["GITLAB_BRANCH"] = "bot-branch-name"
os.environ["GITLAB_BASE_BRANCH"] = "main"

# This example also requires an OpenAI API key
os.environ["OPENAI_API_KEY"] = ""

In [5]:
llm = OpenAI(temperature=0)
gitlab = GitLabAPIWrapper()
toolkit = GitLabToolkit.from_gitlab_api_wrapper(gitlab)
agent = initialize_agent(
    toolkit.get_tools(), llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [8]:
agent.run(
    "You have the software engineering capabilities of a Google Principle engineer. You are tasked with completing issues on a gitlab repository. Please look at the open issues and complete them by creating merge requests that solve the issues."
)



> Entering new AgentExecutor chain...
 I need to look at the open issues and figure out how to solve them.
Action: Get Issues
Action Input: N/A
Observation: Found 1 issues:
[{'title': 'Add tic-tac-toe game', 'number': 15}]
Thought: I need to look at the details of this issue to figure out how to solve it.
Action: Get Issue
Action Input: 15
Observation: {"title": "Add tic-tac-toe game", "body": "Create a tic-tac-toe game using HTML, CSS, and JavaScript. Create a new file called game.html and store the code there.", "comments": "[]"}
Thought: I need to create the game.html file and add the code.
Action: Create File
Action Input: game.html

test contents
Observation: Created file game.html
Thought: I need to add the code to the game.html file.
Action: Update File
Action Input: game.html

OLD <<<<
test contents
>>>> OLD
NEW <<<<
<html>
  <head>
    <title>Tic-Tac-Toe</title>
  </head>
  <body>
    <h1>Tic-Tac-Toe</h1>
    <div id="game">
      <!-- game board goes here -->
    </div>
  <

'I have created a merge request with number 12 that solves issue 15.'